# Pytorch
- 구글의 tensorflow와 유사한 딥러닝 라이브러리
- 페이스북 인공지능 연구팀에 의해 주로 개발
- torch
    - 텐서 변환 및 다양한 수학 함수와 클래스가 포함되어 있다.
    - 텐서(Tensor)란?
        - 스칼라, 벡터, 행렬, 다차원 배열등의 다양한 모양의 숫자들을 모아놓은 구조
        - 다차원의 배열을 통칭

- torch.nn
    - 신경망을 구축하기 위한 레이어(층), 활성화 함수, 손실함수 등에 대한 함수와 클래스가 포함되어 있다.
- torch.utils.data
    - 미니 배치 학습을 위한 데이터셋 구성관련 함수와 클래스가 포함되어 있다.
- torch.optim
    - optimizer 관련 함수와 클래스가 포함되어 있다.
- https://pytorch.org/

# 타이타닉 데이터셋을 pytorch로 학습 시켜보기
1. 데이터 전처리
2. 학습 데이터를 텐서로 변경
3. 인공 신경망 모델 객체 생성
4. 하이퍼파라미터(손실함수 및 옵티마이저 등 선택)
5. 학습 및 테스트 예측 loop 구현

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# from google.colab import drive
# drive.mount('/content/drive')

DATA_PATH = "../data/"

SEED = 42  # 시드값

# 데이터 블러오기
train = pd.read_csv(f"{DATA_PATH}titanic_train.csv")  # 학습데이터
test = pd.read_csv(f"{DATA_PATH}titanic_test.csv")  # 테스트 데이터

# 결측치 처리
age_mean = train["age"].mean()
fare_median = train["fare"].median()
cabin_unk = "UNK"
embarked_mode = train["embarked"].mode()[0]
train["age"] = train["age"].fillna(age_mean)
train["cabin"] = train["cabin"].fillna(cabin_unk)
test["age"] = test["age"].fillna(age_mean)
test["fare"] = test["fare"].fillna(fare_median)
test["cabin"] = test["cabin"].fillna(cabin_unk)
test["embarked"] = test["embarked"].fillna(embarked_mode)

# 특성으로 사용할 변수 선택
cols = ["age", "sibsp", "parch", "fare", "pclass", "gender", "embarked"]
train_ft = train[cols].copy()
test_ft = test[cols].copy()

# 범주형 변수 원핫인코딩
cols = ["gender", "embarked"]
enc = OneHotEncoder(handle_unknown="ignore")
enc.fit(train[cols])
tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(), columns=enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft, tmp], axis=1).drop(columns=cols)
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(), columns=enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft, tmp], axis=1).drop(columns=cols)

# Min-Max Scaling
scaler = MinMaxScaler()
scaler.fit(train_ft)
train_ft = scaler.transform(train_ft)
test_ft = scaler.transform(test_ft)

# 정답 데이터
target = train["survived"].to_numpy()

train_ft.shape, test_ft.shape, target.shape

((916, 10), (393, 10), (916,))

In [ ]:
train_ft

array([[0.88726043, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.42377552, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.36114243, 0.375     , 0.11111111, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.44882876, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.473882  , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.23587624, 0.125     , 0.11111111, ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
target

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

# 손실함수(loss function)

## 회귀문제에서의 손실함수

In [ ]:
import torch

- 회귀 문제에서 정답데이터는 2차원 형식이어야 하고, 데이터 타입은 float32 이어야 한다.

In [ ]:
torch.manual_seed(SEED)
x = torch.randn(2,3) # 2개의 샘플, 3개의 피처
y = torch.randn(2,1) # 2개의 정답 데이터, 정답데이터는 2차원 형식이어야 함

In [ ]:
torch.manual_seed(SEED)
model = torch.nn.Linear(x.shape[1], 1) # 선형회귀

In [ ]:
pred = model(x)
pred

tensor([[0.7090],
        [0.1192]], grad_fn=<AddmmBackward0>)

- MSE

In [ ]:
loss_fn = torch.nn.MSELoss() # 손실을 계산할 수 있는 손실함수 객체가 반환됨
loss = loss_fn(pred, y) # 손실함수에는 첫 번째 인수로 예측값, 두 번째 인수로 정답값 전달
loss

tensor(1.4105, grad_fn=<MseLossBackward0>)

In [ ]:
loss.backward() # 역전파

- MAE

In [ ]:
loss_fn = torch.nn.L1Loss()
loss = loss_fn(pred, y)
loss

tensor(1.1282, grad_fn=<MeanBackward0>)

In [ ]:
# loss.backward()

## 분류문제에서의 손실함수

- BCE
    - 이진 분류 문제에서 사용되는 손실함수

In [ ]:
x

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]])

- 이진 분류 문제에서 정답데이터는 2차원 형식이어야 하고, 데이터 타입은 float32 이어야 한다.

In [ ]:
y = torch.Tensor([0,1]).view(-1,1)
y

tensor([[0.],
        [1.]])

In [ ]:
model = torch.nn.Linear(x.shape[1], 1)
pred = model(x)
pred

tensor([[0.2456],
        [0.2315]], grad_fn=<AddmmBackward0>)

- 정의한 모델의 출력이 시그모이드 함수를 통과한 경우

In [ ]:
sig = torch.nn.Sigmoid() # 시그모이드 함수를 반환하는 클래스
loss_fn = torch.nn.BCELoss()
loss_fn(sig(pred), y)

tensor(0.7038, grad_fn=<BinaryCrossEntropyBackward0>)

- 정의한 모델의 출력이 시그모이드 함수를 통과하지 않은 경우*

In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()
loss_fn(pred, y)

tensor(0.7038, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

- CE(Cross Entropy)
    - 다중 분류 문제에서 사용되는 손실함수

In [ ]:
x = torch.randn(4,2) # 4개의 샘플 2개의 피처
x

tensor([[-1.1229, -0.1863],
        [ 2.2082, -0.6380],
        [ 0.4617,  0.2674],
        [ 0.5349,  0.8094]])

- 다중 분류 문제에서의 정답데이터는 1차원 형태여야 하고, 데이터타입은 int64 형태여야 함

In [ ]:
y = torch.tensor([0,1,2,1], dtype=torch.int64)
y.shape, y.dtype

(torch.Size([4]), torch.int64)

In [ ]:
model = torch.nn.Linear(x.shape[1], 3)
pred = model(x)
pred

tensor([[ 1.3261, -0.2972,  0.6011],
        [-0.3860, -0.7703,  1.1389],
        [ 0.2938, -0.8065,  0.3900],
        [ 0.0762, -1.0516,  0.0164]], grad_fn=<AddmmBackward0>)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(pred, y)

tensor(1.3699, grad_fn=<NllLossBackward0>)

# 옵티마이저(Optimizer)
- pytorch 의 옵티마이저의 주요 파라미터
    - 첫 번째 인수로 모델의 파라미터 전달
    - lr: 학습률
    - weight_decay: 가중치 감소(L2 정규화)를 적용하는데 사용되는 계수
        - 기본값은 0: 가중치 감소 x

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7c40a1376810>

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
optim.step() # 가중치 업데이트

## momentum

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Adagrad

In [ ]:
optim = torch.optim.Adagrad(model.parameters())

## RMSprop

In [ ]:
optim = torch.optim.RMSprop(model.parameters())

## Adam*

In [ ]:
optim = torch.optim.Adam(model.parameters())

In [ ]:
# 역전파 후
optim.step()

# 입력 데이터 텐서로 변환

- 일반적으로 입력으로 들어가는 텐서의 dtype 은 float32

In [ ]:
x_train = torch.Tensor(train_ft)
x_train.shape, x_train.dtype

(torch.Size([916, 10]), torch.float32)

# 정답 데이터 텐서로 변환

In [ ]:
y_train = torch.Tensor(target.reshape(-1,1))
y_train.shape, y_train.dtype

(torch.Size([916, 1]), torch.float32)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# 손실함수 객체 생성

In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()

# 학습 loop 구현
- 에폭 100

In [ ]:
torch.manual_seed(SEED)
model = torch.nn.Linear(x_train.shape[1], 1).to(device)
optimizer = torch.optim.Adam(model.parameters())

model.train() # 학습모드
for i in range(100):
    pred = model(x_train.to(device)) # 예측
    loss = loss_fn(pred, y_train.to(device)) # 손실 계산 후 손실 텐서 반환

    optimizer.zero_grad() # 기울기 0으로 초기화
    loss.backward() # 역전파
    optimizer.step() # 가중치 업데이트

    print(f"epoch {i+1} loss: {loss.item()}")

epoch 1 loss: 0.6774646639823914
epoch 2 loss: 0.6765611171722412
epoch 3 loss: 0.6756607890129089
epoch 4 loss: 0.6747637987136841
epoch 5 loss: 0.6738703846931458
epoch 6 loss: 0.6729801893234253
epoch 7 loss: 0.6720936894416809
epoch 8 loss: 0.6712106466293335
epoch 9 loss: 0.6703311800956726
epoch 10 loss: 0.6694554686546326
epoch 11 loss: 0.6685833930969238
epoch 12 loss: 0.6677150130271912
epoch 13 loss: 0.6668505668640137
epoch 14 loss: 0.6659897565841675
epoch 15 loss: 0.6651329398155212
epoch 16 loss: 0.6642799973487854
epoch 17 loss: 0.6634309887886047
epoch 18 loss: 0.6625859141349792
epoch 19 loss: 0.6617448329925537
epoch 20 loss: 0.6609077453613281
epoch 21 loss: 0.660074770450592
epoch 22 loss: 0.6592457890510559
epoch 23 loss: 0.6584208607673645
epoch 24 loss: 0.6576001048088074
epoch 25 loss: 0.6567834615707397
epoch 26 loss: 0.6559708714485168
epoch 27 loss: 0.6551624536514282
epoch 28 loss: 0.6543582081794739
epoch 29 loss: 0.6535580158233643
epoch 30 loss: 0.6527620

In [ ]:
x_test = torch.Tensor(test_ft)
x_test.shape, x_test.dtype

(torch.Size([393, 10]), torch.float32)

In [ ]:
model.eval() # 평가모드
sig = torch.nn.Sigmoid()

with torch.no_grad(): # 예측 과정에서 경사추적 할 필요 없기 때문에 with 문 내에서 예측 수행
    pred = model(x_test.to(device)) # 실수 예측값 반환
    pred = sig(pred) # 예측 확률로 변환
    pred = pred.to("cpu").numpy() # cpu 이동 후 ndarray 변환

pred

array([[0.43729886],
       [0.4247257 ],
       [0.37824133],
       [0.6071703 ],
       [0.60195106],
       [0.5011371 ],
       [0.40200987],
       [0.48077458],
       [0.48190418],
       [0.6115566 ],
       [0.39849135],
       [0.6071784 ],
       [0.53903604],
       [0.5178673 ],
       [0.5789724 ],
       [0.38065284],
       [0.42397705],
       [0.5048228 ],
       [0.49144453],
       [0.3806299 ],
       [0.38200328],
       [0.5545891 ],
       [0.6072311 ],
       [0.37788776],
       [0.3848875 ],
       [0.551164  ],
       [0.39494857],
       [0.43791315],
       [0.42235377],
       [0.5258509 ],
       [0.48895422],
       [0.48309287],
       [0.36441907],
       [0.5060695 ],
       [0.4836451 ],
       [0.52307546],
       [0.62173694],
       [0.42235377],
       [0.37945077],
       [0.3869444 ],
       [0.5046821 ],
       [0.59152395],
       [0.38247728],
       [0.39811417],
       [0.48201627],
       [0.38199148],
       [0.41835648],
       [0.508

In [ ]:
threshold = 0.5
(pred > threshold).astype(int)

array([[0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
    